In [146]:
import numpy as np

#import pandas sebagai library pengolahan data
import pandas as pd

# load algoritma untuk kasus classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier

# load fungsi split dataset
from sklearn.model_selection import train_test_split

#load fungsi preprocessor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

#import metric yang digunakan pada kaggle
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, roc_auc_score

## Exploratory Data

In [121]:
# load data set yang disediakan

train_df = pd.read_csv(r"C:\Users\Teuku Raja\OneDrive - Kemenkeu\IRVII\PusKU\dataset\File-File Pendukung Kompetisi Kaggle Pelatihan Python\Ujian Praktikum - Penjaluran\data_training.csv",
                       sep=";")
test_df = pd.read_csv(r"C:\Users\Teuku Raja\OneDrive - Kemenkeu\IRVII\PusKU\dataset\File-File Pendukung Kompetisi Kaggle Pelatihan Python\Ujian Praktikum - Penjaluran\data_testing.csv",
                      sep=";")

# cek dataset training
train_df.head()

,DocsDate,ImportDuty_IDR,CIF,Valuta_CIF,Using_Intermediaries,Using_TradeAgreement,COuntryOfOrigin,HSCODE,CustChannel
0,09/10/2022,254458953,170458,USD,0,0,TH,8708,Green
1,02/05/2022,1719402722,1237513,USD,0,0,MY,4016,Green
2,02/04/2022,662630024,40640204,JPY,0,0,JP,7318,Green
3,08/05/2022,7443742,1159783,JPY,1,0,CN,8443,Green
4,02/08/2022,20795848,207958981,IDR,0,0,MX,8708,Green


In [122]:
# cek dataset testing

test_df.head()

,ID,DocsDate,ImportDuty_IDR,CIF,Valuta_CIF,Using_Intermediaries,Using_TradeAgreement,COuntryOfOrigin,HSCODE
0,C1,02/01/2022,28212800050,41584240,USD,0,0,NZ,206
1,C2,12/11/2022,259418090,13014510,JPY,0,0,JP,4016
2,C3,16/11/2022,2597872060,1413620,USD,0,0,TH,7318
3,C4,03/09/2022,11550943500,15789680,USD,1,1,CN,8708
4,C5,04/04/2022,1871500400,95652240,JPY,1,0,JP,3926


## Data Preprocessing

In [123]:

# Replace commas in numeric columns
train_df['CIF'] = train_df['CIF'].replace({',': ''}, regex=True)

# Convert numeric columns to numeric type
train_df['CIF'] = train_df['CIF'].apply(pd.to_numeric, errors='coerce')

# imput semua missing value dengan angka 0
train_df = train_df.dropna()
test_df = test_df.dropna()

In [124]:
# penentuan prediktor dan target

# prediktor data training 
x_train = train_df[['ImportDuty_IDR', 'CIF']]

# label data training
y_train = train_df.iloc[:,-1]

# prediktor untuk prediksi dataset testing
x_test = test_df[['ImportDuty_IDR', 'CIF']]

In [125]:
# ## Anda mungkin perlu melakukan encoding kolom categorical menjadi numeric
# label_encoder1 = LabelEncoder()
# x_train['Valuta_CIF'] = label_encoder1.fit_transform(x_train['Valuta_CIF'])

# label_encoder2 = LabelEncoder()
# x_train['COuntryOfOrigin'] = label_encoder2.fit_transform(x_train['COuntryOfOrigin'])


### Splitting Dataset
pendekatan hold out, jika CV tidak perlu karena didalamnya sudah ada split evaluation dataset

In [126]:
# split data training dan evaluation
x_train, x_eval, y_train, y_eval = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [127]:
# cek dimensi seluruh dataset
print(x_train.shape, x_eval.shape, x_test.shape)

(390967, 2) (68995, 2) (39996, 2)


In [128]:
print(y_train.shape, y_eval.shape)

(390967,) (68995,)


## Model Building
### Fitting / Training Model

In [147]:
#untuk contoh, digunakan algoritma paling simple yaitu linear regression
model = DummyClassifier(strategy="most_frequent")
model.fit(x_train, y_train)

DummyClassifier(strategy='most_frequent')

### Evaluating Model

In [149]:
# mengevaluasi model dengan data training (tidak disarankan karena akan misleading)
y_pred = model.predict(x_eval)
accuracy = accuracy_score(y_eval, y_pred)
accuracy

0.9141821871150083

## Finishing
### Make Prediction

In [150]:
submit_predict = model.predict(x_test)

### Export File for Submission

In [152]:
test_df['CustChannel'] = submit_predict
submit_df = test_df[['ID','CustChannel']].copy()
submit_df.to_csv('submit.csv', sep=",", index=False)